In [1]:
import pandas as pd 
import numpy as np

In [2]:
df = pd.read_csv('GA_features.csv')
df

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,county,tweets,contribution,n_poll,frac_unem,frac_votes,total_votes,...,frac_voted_A,frac_voted_B,frac_voted_C,frac_voted_D,frac_voted_E,frac_voted_F,frac_voted_G,total_registers,latitude,longitude
0,0,0,0,APPLING,0,235818,0,0.160193,0.695728,8403,...,0.155540,0.783292,0.004165,0.012972,0.001071,0.004046,0.038915,12078,31.7492,-82.2889
1,1,1,1,ATKINSON,0,93026,0,0.117003,0.649241,3167,...,0.187243,0.729081,0.000947,0.050837,0.000947,0.003789,0.027155,4878,31.2971,-82.8800
2,2,2,2,BACON,0,173074,0,0.115665,0.672325,4674,...,0.085794,0.858151,0.002353,0.010270,0.000642,0.003637,0.039153,6952,31.5537,-82.4527
3,3,3,3,BAKER,0,72226,0,0.179682,0.682852,1561,...,0.383088,0.570788,0.006406,0.006406,0.001281,0.002562,0.029468,2286,31.3262,-84.4447
4,4,4,4,BALDWIN,2,927608,0,0.173954,0.673200,18342,...,0.374932,0.572239,0.008123,0.007796,0.000872,0.005670,0.030367,27246,33.0693,-83.2496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,154,154,154,WHITFIELD,0,4216384,1,0.070532,0.660559,36955,...,0.034745,0.749127,0.009552,0.127588,0.001380,0.009796,0.067812,55945,34.8056,-84.9672
155,155,155,155,WILCOX,0,101176,0,0.182849,0.690566,3294,...,0.230419,0.740437,0.002429,0.007286,0.000607,0.003643,0.015179,4770,31.9729,-83.4323
156,156,156,156,WILKES,0,142526,0,0.206583,0.703740,5024,...,0.344944,0.605494,0.004976,0.004379,0.001393,0.005772,0.033041,7139,33.7819,-82.7432
157,157,157,157,WILKINSON,0,92444,0,0.163246,0.711561,4776,...,0.383585,0.581449,0.001675,0.002094,0.001466,0.003769,0.025963,6712,32.8024,-83.1712


In [4]:
# List of counties to set n_poll = 1
target_counties = [
    'BAKER', 'CALHOUN', 'CHARLTON', 'CLAY', 'CLINCH', 'DODGE', 'EARLY', 'GLASCOCK',
    'HANCOCK', 'LINCOLN', 'MONTGOMERY', 'QUITMAN', 'RANDOLPH', 'SCREVEN', 'STEWART',
    'TALBOT', 'TALIAFERRO', 'WILCOX', 'WILKINSON', 'WORTH'
]

# Update n_poll: 1 for target counties, 0 for others
df['n_poll'] = df['county'].apply(lambda x: 1 if x in target_counties else 0)

# Display the modified DataFrame
df

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,county,tweets,contribution,n_poll,frac_unem,frac_votes,total_votes,...,frac_voted_A,frac_voted_B,frac_voted_C,frac_voted_D,frac_voted_E,frac_voted_F,frac_voted_G,total_registers,latitude,longitude
0,0,0,0,APPLING,0,235818,0,0.160193,0.695728,8403,...,0.155540,0.783292,0.004165,0.012972,0.001071,0.004046,0.038915,12078,31.7492,-82.2889
1,1,1,1,ATKINSON,0,93026,0,0.117003,0.649241,3167,...,0.187243,0.729081,0.000947,0.050837,0.000947,0.003789,0.027155,4878,31.2971,-82.8800
2,2,2,2,BACON,0,173074,0,0.115665,0.672325,4674,...,0.085794,0.858151,0.002353,0.010270,0.000642,0.003637,0.039153,6952,31.5537,-82.4527
3,3,3,3,BAKER,0,72226,1,0.179682,0.682852,1561,...,0.383088,0.570788,0.006406,0.006406,0.001281,0.002562,0.029468,2286,31.3262,-84.4447
4,4,4,4,BALDWIN,2,927608,0,0.173954,0.673200,18342,...,0.374932,0.572239,0.008123,0.007796,0.000872,0.005670,0.030367,27246,33.0693,-83.2496
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,154,154,154,WHITFIELD,0,4216384,0,0.070532,0.660559,36955,...,0.034745,0.749127,0.009552,0.127588,0.001380,0.009796,0.067812,55945,34.8056,-84.9672
155,155,155,155,WILCOX,0,101176,1,0.182849,0.690566,3294,...,0.230419,0.740437,0.002429,0.007286,0.000607,0.003643,0.015179,4770,31.9729,-83.4323
156,156,156,156,WILKES,0,142526,0,0.206583,0.703740,5024,...,0.344944,0.605494,0.004976,0.004379,0.001393,0.005772,0.033041,7139,33.7819,-82.7432
157,157,157,157,WILKINSON,0,92444,1,0.163246,0.711561,4776,...,0.383585,0.581449,0.001675,0.002094,0.001466,0.003769,0.025963,6712,32.8024,-83.1712


In [5]:
df.to_csv('GA_features.csv')